In [ ]:
!pip install --upgrade gspread oauth2client
!pip install pandas

In [78]:
# Auto - automaticly export JSON to Google Sheets
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [79]:
# Manual - generated .csv, imported manually to Google Sheets
import pandas as pd

In [80]:
# Auto
# API setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
# Replace serviceacc.json to own credentials json
creds = ServiceAccountCredentials.from_json_keyfile_name("serviceacc.json", scope)
client = gspread.authorize(creds)

In [81]:
# Auto
sheet_id = "1C9Mc4-x8VUvbOG1ZstPVi069GxXjxWylqh3Ujow34Q4"
sheet_name = "Unit Rate"

# Open sheets
sheet = client.open_by_key(sheet_id)
worksheet = sheet.worksheet(sheet_name)

In [82]:
# Define JSON path (replace your own URL)
df = pd.read_json("https://www.abc.com.my/FundPriceUT/GetAllUTFundPriceByDate?date=20-11-2023")


In [83]:
def splitcol(r):
  resultData = r["ResultData"]
  r["date"] = resultData["Date"]
  r["fundcode"] = resultData["FundAbbr"]
  r["fund"] = resultData["FundDesc"]
  r["nav"] = resultData["Nav"]
  r["change"] = resultData["ChangePercent"]
  r["exposure"] = resultData["ForeignExposureCode"]
  return r

In [84]:
ndf = df.apply(splitcol, axis=1).drop(columns=["ResponseCode","Message","ShowMessage","RedirectPath","ResultData"])
#                                drop("ResultData", axis=1)
ndf.head(10)

,date,fundcode,fund,nav,change,exposure
0,2023-11-20T00:00:00,PAVGEF,PUBLIC ADVANTAGE GROWTH EQUITY FUND,0.2682,0.41%,MIX
1,2023-11-20T00:00:00,PAGF,PUBLIC AGGRESSIVE GROWTH FUND,0.5929,0.19%,MIX
2,2023-11-20T00:00:00,PASGF,PUBLIC ASEAN GROWTH FUND,0.2929,-0.14%,FOREIGN
3,2023-11-20T00:00:00,PAIF,PUBLIC ASIA ITTIKAL FUND,0.3768,0.88%,FOREIGN
4,2023-11-20T00:00:00,PAPA40F,PUBLIC ASIA PACIFIC ALPHA-40 FUND,0.2299,0.70%,FOREIGN
5,2023-11-20T00:00:00,PAPFF,PUBLIC ASIA PACIFIC FOCUS FUND,0.2159,0.84%,FOREIGN
6,2023-11-20T00:00:00,PAUEF,PUBLIC AUSTRALIA EQUITY FUND,0.2579,0.86%,FOREIGN
7,2023-11-20T00:00:00,PBF,PUBLIC BALANCED FUND,0.6764,0.22%,MIX
8,2023-11-20T00:00:00,PBOND,PUBLIC BOND FUND,0.9450,0.06%,MIX
9,2023-11-20T00:00:00,PCASEF,PUBLIC CHINA ACCESS EQUITY FUND,1.0078,0.97%,FOREIGN


In [85]:
# Manual
ndf.to_csv("data.csv")

In [86]:
# Auto
worksheet.clear()
# Export DataFrame
set_with_dataframe(worksheet, ndf)